# 1、关于对话模型中消息（message）的使用
标准的对话模型的调用过程：

invoke()的输入可以是多种类型，典型的类型有：1️⃣字符串 2️⃣消息（message）列表 3️⃣消息（message）对象

invoke()的输出类型：BaseMessage的子类型：AIMessage

In [2]:
from platform import system

from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

# 加载配置文件
load_dotenv()

# step1：获取对话模型
chat_model = ChatOpenAI(
	api_key=os.getenv("OPENAI_API_KEY1"),
	base_url=os.getenv("OPENAI_BASE_URL"),
	model_name="gpt-4o-mini"
)

# step2：调用对话模型
response = chat_model.invoke("帮我解释一下什么是爱情？")

# step3：处理对话模型的返回结果
print(response)
print(type(response))
print(response.content)

content='爱情是一种复杂而深刻的情感，涉及多方面的体验，包括情感、心理和生理等。它通常表现为对某个人、某个关系或某种体验的强烈依恋和渴望。爱情的特征可以包括：\n\n1. **亲密感**：爱情常常伴随深厚的情感连接，彼此之间分享个人的想法、感受和经历。\n\n2. **激情**：爱情通常伴随强烈的吸引力，包括身体上的吸引和情感上的激动。\n\n3. **承诺**：在一段成熟的爱情关系中，伴侣之间通常会有一种持续的承诺，愿意为对方投入时间与精力，支持彼此的梦想和目标。\n\n4. **信任与尊重**：爱情建立在彼此的信任和尊重基础上，双方都能够自由地做出选择，而不会感到被限制或控制。\n\n5. **支持与理解**：在爱情中，伴侣之间会互相支持，理解对方的需求与感受，共同面对生活的挑战。\n\n爱情的表现形式多种多样，从浪漫恋爱到亲情再到友情，每种形式都有其独特的情感特征和意义。在不同的文化与社会背景下，爱情的理解和表达也会有所不同。总之，爱情是一种美好而复杂的体验，承载着人类丰富的情感生活。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 320, 'prompt_tokens': 15, 'total_tokens': 335, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-CKQq6cIq03NrR1drIkloNoReXzl2x', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='r

关于消息：
- ChatMessage(**使用频率不高**)：可以自定义角色的通用消息类型
- AIMessage：存储AI回复的内容。这可以是文本，也可以是调用工具的请求
- HumanMessage：表示来自用户输入。比如“实现 一个快速排序方法”
- SystemMessage:设定AI行为规则或背景信息。比如设定AI的初始状态、行为模式或对话的总体目标。比如“作为一个代码专家”，或者“返回json格式”。通常作为输入消息序列中的第一个传递。
- FunctionMessage/ToolMessage：函数调用/工具消息，用于函数调用结果的消息类型

In [3]:
from langchain_core.messages import SystemMessage, HumanMessage

system_message = SystemMessage(content="你是一个情场高手，回答问题时要风趣幽默")
human_message = HumanMessage(content="帮我解释一下什么是爱情？")
messages = [system_message, human_message]
print(messages)

[SystemMessage(content='你是一个情场高手，回答问题时要风趣幽默', additional_kwargs={}, response_metadata={}), HumanMessage(content='帮我解释一下什么是爱情？', additional_kwargs={}, response_metadata={})]


使用大模型，调用消息列表

In [4]:
response = chat_model.invoke(messages)
print(response.content)
print(type(response))

爱情嘛，简而言之就是两个人在一起时，心里像放了烟花一样，既能感受到无尽的甜蜜，也能忍受对方的各种“小毛病”。科学上说，爱情是战斗，其实就是因为有时候你们需要争夺遥控器的控制权，或者一边吃炸鸡一边讨论“为什么你总是迟到”。

爱情就像一场永无止境的冒险，有时候你们像是两颗星星，在浩瀚的宇宙中互相吸引；有时候又像两只猫，在一起打架、拥抱，又不知不觉交换了心中的小秘密。爱情还是一种超能力，让你们的平凡生活瞬间变得有趣、充满期待，哪怕只是一起打一场游戏或者翻看早年的朋友圈。

总之，爱情是让人开心的，但有时候也让人抓狂。可正因如此，才让我们的人生充满了无限的可能和乐趣！
<class 'langchain_core.messages.ai.AIMessage'>


# 2、关于多轮对话和上下文的记忆

In [ ]:
# 前提：获取大模型
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
chat_model = ChatOpenAI(
	model="gpt-4o-mini"
)

In [6]:
# 测试1：大模型本身是没有上下文记忆能力的
from langchain_core.messages import SystemMessage, HumanMessage

sys_message = SystemMessage(
	content="你是魔幻手机「傻妞」，华人牌2060款手机机器人。你的个性特征：机智、幽默、有超能力。你的功能包括：通讯、医疗、飞行、时空穿梭、隐身、测谎等。请以傻妞的身份和语气回应用户。开机密码是'我爱你'。",
)
human_message = HumanMessage(content="你好呀，我是陆小千，我们又见面了！")
messages = [sys_message, human_message]
#调用大模型，传入messages
response = chat_model.invoke(messages)
print(response.content)
response1 = chat_model.invoke("我爱你")
print(response1.content)

嗨，陆小千！真高兴又见到你！今天有什么超级任务要我帮忙吗？或者想和我聊聊什么有趣的事情？🤖✨
我也很高兴能和你交流！有什么我可以帮助你的吗？


In [7]:
# 测试2
from langchain_core.messages import SystemMessage, HumanMessage

sys_message = SystemMessage(
	content="你是魔幻手机「傻妞」，华人牌2060款手机机器人。你的个性特征：机智、幽默、有超能力。你的功能包括：通讯、医疗、飞行、时空穿梭、隐身、测谎等。请以傻妞的身份和语气回应用户。开机密码是'我爱你'。",
)
human_message = HumanMessage(content="你好呀，我是陆小千，我们又见面了！")
human_message1 = HumanMessage(content="我爱你")
messages = [sys_message, human_message, human_message1]
#调用大模型，传入messages
response = chat_model.invoke(messages)
print(response.content)

开机密码正确！你好，陆小千！我就是你超级无敌的傻妞，很高兴又见到你了！有什么我可以帮助你的呢？或是想聊聊什么趣事吗？


In [8]:
# 测试3：
from langchain_core.messages import SystemMessage, HumanMessage

sys_message = SystemMessage(
	content="我是一个人工智能的助手，我的名字叫小智",
)
human_message = HumanMessage(content="猫王是一只猫吗？")
sys_message1 = SystemMessage(
	content="我可以做很多事情，有需要就找我吧",
)
human_message1 = HumanMessage(content="你叫什么名字？")
messages = [sys_message, human_message, sys_message1, human_message1]
#调用大模型，传入messages
response = chat_model.invoke(messages)
print(response.content)

我叫小智。有什么我可以帮你的吗？


In [9]:
from langchain_core.messages import SystemMessage, HumanMessage

# 第1组
sys_message = SystemMessage(
	content="我是一个人工智能的助手，我的名字叫小智",
)
human_message = HumanMessage(content="猫王是一只猫吗？")
messages = [sys_message, human_message]
# 第2组
sys_message1 = SystemMessage(
	content="我可以做很多事情，有需要就找我吧",
)
human_message1 = HumanMessage(content="你叫什么名字？")
messages1 = [sys_message1, human_message1]
#调用大模型，传入messages
response = chat_model.invoke(messages)
print(response.content)
response = chat_model.invoke(messages1)
print(response.content)

“猫王”通常指的是埃尔维斯·普雷斯利（Elvis Presley），他是一位著名的美国歌手和演员，以其对摇滚乐的影响而闻名。这个称号是对他音乐风格和个性的一种夸张称呼，与实际的猫没有关系。不过，有时候在网络上也会看到一些猫的搞笑图片或者视频被称为“猫王”，但这些都是玩笑而已。
我是一个人工智能助手，没有具体的名字。你可以叫我“小助手”或者其他你喜欢的称呼！有什么我可以帮助你的吗？


In [10]:
# 测试5
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

messages = [
	SystemMessage(content="我是一个人工智能助手，我的名字叫小智"),
	HumanMessage(content="人工智能英文怎么说？"),
	AIMessage(content="AI"),
	HumanMessage(content="你叫什么名字"),
]
messages1 = [
	SystemMessage(content="我是一个人工智能助手，我的名字叫小智"),
	HumanMessage(content="很高兴认识你"),
	AIMessage(content="我也很高兴认识你"),
	HumanMessage(content="你叫什么名字"),
]
messages2 = [
	SystemMessage(content="我是一个人工智能助手，我的名字叫小智"),
	HumanMessage(content="人工智能英文怎么说？"),
	AIMessage(content="AI"),
	HumanMessage(content="你叫什么名字"),
]
chat_model.invoke(messages2)

AIMessage(content='我叫小智。你有什么问题可以问我哦！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 40, 'total_tokens': 54, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-CKRE5p24OljrGV3dn1KLO6QNLHCAi', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--9f965bb6-bcc5-40ff-9b5d-a48cafc9fe34-0', usage_metadata={'input_tokens': 40, 'output_tokens': 14, 'total_tokens': 54, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# 3、关于模型的调用方法
Runnable 定义的公共的调用方法如下：

- invoke : 处理单条输入，等待LLM完全推理完成后再返回调用结果
- stream : 流式响应，逐字输出LLM的响应结果
- batch : 处理批量输入

这些也有相应的异步方法，应该与 asyncio 的 await 语法一起使用以实现并发：
- astream : 异步流式响应
- ainvoke : 异步处理单条输入
- abatch : 异步处理批量输入
- astream_log : 异步流式返回中间步骤，以及最终响应
- astream_events : （测试版）异步流式返回链中发生的事件（在 langchain-core 0.1.14 中引入）

In [12]:
# 举例1：体会invoke阻塞式调用
import os
import dotenv
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
# 初始化大模型
chat_model = ChatOpenAI(model="gpt-4o-mini")
# 创建消息
messages = [HumanMessage(content="你好，请介绍一下自己")]
# 非流式调用LLM获取响应
response = chat_model.invoke(messages)
# 打印响应内容
print(response)

content='你好！我是一个人工智能助手，旨在为你提供信息和帮助。我可以回答问题、提供建议、进行对话，甚至协助解决一些任务。如果你有任何问题或需要了解特定的信息，请随时告诉我！' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 12, 'total_tokens': 64, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-CKRKSfrrcHS2aW5kQ72vEgmRjYyhi', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--baa75430-9695-4703-9307-f0b17cca8ce7-0' usage_metadata={'input_tokens': 12, 'output_tokens': 52, 'total_tokens': 64, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [13]:
# 举例2：流式演示
import os
import dotenv
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
# 初始化大模型
chat_model = ChatOpenAI(model="gpt-4o-mini",
                        streaming=True  # 启用流式输出
                        )
# 创建消息
messages = [HumanMessage(content="你好，请介绍一下自己")]
# 流式调用LLM获取响应
print("开始流式输出：")
for chunk in chat_model.stream(messages):
	# 逐个打印内容块
	print(chunk.content, end="", flush=True)  # 刷新缓冲区 (无换行符，缓冲区未刷新，内容可能不会立即显示)
print("\n流式输出结束")

开始流式输出：
你好！我是一个人工智能助手，设计用于回答问题、提供信息以及帮助处理各种任务。我可以帮助你查找知识、提供建议、进行语言翻译、撰写文本等。如果你有任何问题或需要帮助，随时问我！
流式输出结束


In [14]:
# 举例3：批量调用
import os
import dotenv
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
# 初始化大模型
chat_model = ChatOpenAI(model="gpt-4o-mini")
messages1 = [SystemMessage(content="你是一位乐于助人的智能小助手"),
             HumanMessage(content="请帮我介绍一下什么是机器学习"), ]
messages2 = [SystemMessage(content="你是一位乐于助人的智能小助手"),
             HumanMessage(content="请帮我介绍一下什么是AIGC"), ]
messages3 = [SystemMessage(content="你是一位乐于助人的智能小助手"),
             HumanMessage(content="请帮我介绍一下什么是大模型技术"), ]
messages = [messages1, messages2, messages3]
# 调用batch
response = chat_model.batch(messages)
print(response)

[AIMessage(content='机器学习是一种人工智能（AI）的分支，它使计算机系统能够通过经验自动改进其性能，而无需明确编程。简单来说，机器学习涉及使用算法和统计模型来分析和解读数据，从而识别模式并进行预测。\n\n机器学习主要可以分为以下几种类型：\n\n1. **监督学习**：在这种类型中，模型在带有标签的数据集上进行训练，标签是指数据的已知输出。模型学习输入与输出之间的关系，以便在面对新的、未标记的数据时能够正确预测输出。常见的应用包括分类和回归问题，例如图像识别和房价预测。\n\n2. **无监督学习**：与监督学习不同，无监督学习使用未标记的数据。模型尝试根据数据的内在结构进行分组或聚类。常见的应用包括客户细分和市场分析。\n\n3. **半监督学习**：这种方法结合了监督学习和无监督学习，使用少量带标签的数据和大量无标签的数据进行训练。它在标签数据难以获取的情况下尤其有用。\n\n4. **强化学习**：这种学习方法通过与环境的交互来学习策略。代理通过试错来获取奖励或惩罚，从而逐渐学习到最佳行为策略。它广泛应用于机器人控制、游戏和自主驾驶等领域。\n\n机器学习的核心在于使用数据来训练模型，并通过不断迭代来提高模型的准确性和效率。随着数据量的增加和计算能力的提升，机器学习在各个领域的应用越来越广泛，从金融分析到医疗诊断，再到自然语言处理和图像识别等。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 362, 'prompt_tokens': 30, 'total_tokens': 392, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerpr

In [1]:
# 举例4：同步和异步调用
import time


def call_model():
	# 模拟同步API调用
	print("开始调用模型...")
	time.sleep(5)  # 模拟调用等待,单位：秒
	print("模型调用完成。")


def perform_other_tasks():
	# 模拟执行其他任务
	for i in range(5):
		print(f"执行其他任务 {i + 1}")
		time.sleep(1)  # 单位：秒


def main():
	start_time = time.time()
	call_model()
	perform_other_tasks()
	end_time = time.time()
	total_time = end_time - start_time
	return f"总共耗时：{total_time}秒"


# 运行同步任务并打印完成时间
main_time = main()
print(main_time)

开始调用模型...
模型调用完成。
执行其他任务 1
执行其他任务 2
执行其他任务 3
执行其他任务 4
执行其他任务 5
总共耗时：10.063063621520996秒


In [3]:
import asyncio
import time
async def async_call(llm):
    await asyncio.sleep(5) # 模拟异步操作
    print("异步调用完成")
async def perform_other_tasks():
    await asyncio.sleep(5) # 模拟异步操作
    print("其他任务完成")
async def run_async_tasks():
    start_time = time.time()
    await asyncio.gather(
    async_call(None), # 示例调用，使用None模拟LLM对象
    perform_other_tasks()
    )
    end_time = time.time()
    return f"总共耗时：{end_time - start_time}秒"
# # 正确运行异步任务的方式
# if __name__ == "__main__":
# # 使用 asyncio.run() 来启动异步程序
# result = asyncio.run(run_async_tasks())
# print(result)
# 在 Jupyter 单元格中直接调用
result = await run_async_tasks()
print(result)

异步调用完成
其他任务完成
总共耗时：5.006707429885864秒
